In [77]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle 

In [78]:
data = pd.read_csv("/Users/maxkucher/test_api/titanic (1).csv")


In [79]:
data["Sex"].unique()

array(['male', 'female'], dtype=object)

In [80]:
data["Sex"].isnull().sum()

np.int64(0)

In [81]:
data = data.replace({"Sex": {"male": 0, "female": 1}})
data

/var/folders/tl/sxyv3tjs7wlff0fs8dht26pr0000gn/T/ipykernel_59622/2341203803.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({"Sex": {"male": 0, "female": 1}})


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.0000,C148,C


In [82]:
data = data[["Survived", "Sex", "Pclass", "Age", "Fare"]]
data = data.dropna(subset=["Age"])
data

,Survived,Sex,Pclass,Age,Fare
0,0,0,3,22.0,7.2500
1,1,1,1,38.0,71.2833
2,1,1,3,26.0,7.9250
3,1,1,1,35.0,53.1000
4,0,0,3,35.0,8.0500
...,...,...,...,...,...
885,0,1,3,39.0,29.1250
886,0,0,2,27.0,13.0000
887,1,1,1,19.0,30.0000
889,1,0,1,26.0,30.0000


In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    int64  
 1   Sex       714 non-null    int64  
 2   Pclass    714 non-null    int64  
 3   Age       714 non-null    float64
 4   Fare      714 non-null    float64
dtypes: float64(2), int64(3)
memory usage: 33.5 KB


In [84]:
x = data.drop("Survived", axis="columns")
y = data["Survived"]


In [85]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [86]:
def experiment(max_depth: int, min_samples_split: int) -> None:
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split)
    model.fit(x_train, y_train)


    preds = model.predict(x_test)
    acc = accuracy_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)

    print(f"Accuracy: {acc}")
    print(f"Confusion matrix: {cm}")

    report = classification_report(y_test, preds, target_names=["Not survived", "Survived"])
    print(f"Report: {report}")

    with open("model.pkl", "wb") as f:
        pickle.dump(model, f)


max_depth = 5
min_samples_split = 150

experiment(max_depth = max_depth, min_samples_split = min_samples_split)

Accuracy: 0.7342657342657343
Confusion matrix: [[68 13]
 [25 37]]
Report:               precision    recall  f1-score   support

Not survived       0.73      0.84      0.78        81
    Survived       0.74      0.60      0.66        62

    accuracy                           0.73       143
   macro avg       0.74      0.72      0.72       143
weighted avg       0.74      0.73      0.73       143



In [87]:
with open("./model.pkl", "rb") as f:
    model = pickle.load(f)

In [88]:
x_test[:1]

,Sex,Pclass,Age,Fare
684,0,2,60.0,39.0


In [89]:
new_data = pd.DataFrame({
    "Sex": [0],
    "Pclass": [1],
    "Age": [4],
    "Fare": [8.233]
})

In [90]:
prediction = model.predict(new_data)
print(f"Prediction: {prediction.item()}")

Prediction: 0


In [104]:
import requests

def predict_model(data):
    url = "http://127.0.0.1:500/predict_model"

    response = requests.post(url, json=data)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Request failed with status code: {response.status_code}"}
    

data = {
    "Sex": 1,
    "Pclass": 1,
    "Age": 4,
    "Fare": 237.250
}

prediction = predict_model(data)
print(prediction)

{'prediction': 'Survived'}
